## Chargement des données

Les données COCO () sont accessibles notamment via le package `fiftyone` qui permet de sélectionner les images de façon aléatoire. Nous ne sommes intéressés que par les sous-titres des images (_captions_).

La structure d'un jeu de données de cette librairie `fyftyone` n'est pas transparente (elle est prévue pour des interactions via cette librairie). Avec quelques tests, on identifier qu'un jeu de donnée possède une méthode `to_dict()`, dans lequel l'entréee correspondant au tableau de données est `samples`.

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

In [ ]:
coco_images_100 = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    max_samples=100,
    shuffle=True,
    include_id=True,
    seed=5212
)
coco_images_100 = coco_images_100.to_dict()['samples']

 100% |██████|    1.9Gb/1.9Gb [3.2s elapsed, 0s remaining, 544.2Mb/s]      
Extracting annotations to '/root/fiftyone/coco-2017/raw/instances_train2017.json'
 100% |██████████████████| 100/100 [2.3s elapsed, 0s remaining, 46.3 images/s]      
Writing annotations for 100 downloaded samples to '/root/fiftyone/coco-2017/train/labels.json'
Dataset info written to '/root/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'train'
 100% |█████████████████| 100/100 [552.6ms elapsed, 0s remaining, 181.0 samples/s]      
Dataset 'coco-2017-train-100' created
 100% |█████████████████| 100/100 [370.7ms elapsed, 0s remaining, 269.7 samples/s]      


In [ ]:
coco_images_100[1] # notice "coco_id" et "filepath"

{'filepath': '/root/fiftyone/coco-2017/train/data/000000427123.jpg',
 'tags': ['train'],
 'metadata': {'_cls': 'ImageMetadata', 'width': 640, 'height': 426},
 'detections': {'_cls': 'Detections',
  'detections': [{'_id': {'$oid': '61b39fa048cd9efa2de54b62'},
    '_cls': 'Detection',
    'attributes': {},
    'tags': [],
    'label': 'tie',
    'bounding_box': [0.2866875,
     0.4150938967136151,
     0.021453125,
     0.1264319248826291],
    'supercategory': 'accessory',
    'iscrowd': 0},
   {'_id': {'$oid': '61b39fa048cd9efa2de54b63'},
    '_cls': 'Detection',
    'attributes': {},
    'tags': [],
    'label': 'person',
    'bounding_box': [0.1801875,
     0.22840375586854458,
     0.218890625,
     0.7427230046948357],
    'supercategory': 'person',
    'iscrowd': 0},
   {'_id': {'$oid': '61b39fa048cd9efa2de54b64'},
    '_cls': 'Detection',
    'attributes': {},
    'tags': [],
    'label': 'person',
    'bounding_box': [0.400875,
     0.19551643192488263,
     0.20790625000000001,

In [ ]:
import pandas as pd

# format as pandas' data frame
coco_images_100_df = pd.DataFrame( {
    "id"       : [image['coco_id'] for image in coco_images_100 ],
    "url"      : ['http://cocodataset.org/#explore?id=' + str( image['coco_id'] ) for image in coco_images_100 ],
    "filepath" : [image['filepath'] for image in coco_images_100 ],
})

# save in order to prevent later re-downloading and re-generation
coco_images_100_df.to_csv("data/0-coco_df-id-url.csv", index=False)

In [ ]:
coco_images_100_df 

,id,url,filepath
0,393274,http://cocodataset.org/#explore?id=393274,/root/fiftyone/coco-2017/train/data/0000003932...
1,427123,http://cocodataset.org/#explore?id=427123,/root/fiftyone/coco-2017/train/data/0000004271...
2,405324,http://cocodataset.org/#explore?id=405324,/root/fiftyone/coco-2017/train/data/0000004053...
3,300357,http://cocodataset.org/#explore?id=300357,/root/fiftyone/coco-2017/train/data/0000003003...
4,57340,http://cocodataset.org/#explore?id=57340,/root/fiftyone/coco-2017/train/data/0000000573...
...,...,...,...
95,298014,http://cocodataset.org/#explore?id=298014,/root/fiftyone/coco-2017/train/data/0000002980...
96,250225,http://cocodataset.org/#explore?id=250225,/root/fiftyone/coco-2017/train/data/0000002502...
97,484457,http://cocodataset.org/#explore?id=484457,/root/fiftyone/coco-2017/train/data/0000004844...
98,541039,http://cocodataset.org/#explore?id=541039,/root/fiftyone/coco-2017/train/data/0000005410...


## Notation des images par des humains

Pour chacune des 100 images de COCO, un humain (parmi 4) a :

1. utilisé l'URL `http://cocodataset.org/#explore?id=<id>` pour se rendre sur la page correspondant à l'image sur le site de COCO
2. a clicé sur une icone pour obtenir les légendes associées à l'image et a récopié la première dans [un tableur](https://docs.google.com/spreadsheets/d/136t5fMGIjtrhWdjb9FfwfbZy48LRW2Uge9FyRyPsxbk)
3. a proposé deux autres légendes pour l'image, l'une plus ou moins approchante, l'autre grossièrement différente

Enfin, un second humain (toujours parmi les 4 premiers, mais différent de celui qui a proposé les légendes) :

1. a donné un score d'adéquation entre l'image et la légende entre 0 (aucune correspondance) et 1 (description optimale)

Ces données sont disponibles [**ici**](https://docs.google.com/spreadsheets/d/136t5fMGIjtrhWdjb9FfwfbZy48LRW2Uge9FyRyPsxbk) et recensent l'image `id`, l'URL `url`, la personne / l'institution à l'origine de la légende `captionist`, le notateur `assessor` et la note `grade`.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
coco_ratings_300_df = pd.read_csv(
    "data/1-human-grades.csv",
    names=['url','captionist','assessor','caption', 'human_grade'],
    usecols=range(5),
    header=0,
    decimal=','
)
coco_ratings_300_df = coco_ratings_300_df.merge( coco_images_100_df , on='url')
coco_ratings_300_df.to_csv("data/2-coco_df-id-url-captionnist-assessor-grade.csv", index=False)
coco_ratings_300_df

,url,captionist,assessor,caption,grade,id,filepath
0,http://cocodataset.org/#explore?id=393274,COCO,Aymeric,a yellow and white train traveling past a trai...,0.9,393274,/root/fiftyone/coco-2017/train/data/0000003932...
1,http://cocodataset.org/#explore?id=393274,Arthur,Aymeric,several trains parked in a train station,0.3,393274,/root/fiftyone/coco-2017/train/data/0000003932...
2,http://cocodataset.org/#explore?id=393274,Arthur,Aymeric,a river with people nearby,0.0,393274,/root/fiftyone/coco-2017/train/data/0000003932...
3,http://cocodataset.org/#explore?id=427123,COCO,Léo,"three people in formal military uniforms, two ...",1.0,427123,/root/fiftyone/coco-2017/train/data/0000004271...
4,http://cocodataset.org/#explore?id=427123,Arthur,Léo,smiling researchers receiving the Nobel prize,0.5,427123,/root/fiftyone/coco-2017/train/data/0000004271...
...,...,...,...,...,...,...,...
295,http://cocodataset.org/#explore?id=541039,Aymeric,Conrad,smiling black people who say thank you,0.9,541039,/root/fiftyone/coco-2017/train/data/0000005410...
296,http://cocodataset.org/#explore?id=541039,Aymeric,Conrad,Birthday gifts in a box,0.0,541039,/root/fiftyone/coco-2017/train/data/0000005410...
297,http://cocodataset.org/#explore?id=423313,COCO,Arthur,a group of motorcycles parked in front of a bu...,0.9,423313,/root/fiftyone/coco-2017/train/data/0000004233...
298,http://cocodataset.org/#explore?id=423313,Aymeric,Arthur,Mortorcyles hiding small stores,0.7,423313,/root/fiftyone/coco-2017/train/data/0000004233...


In [ ]:
import plotly.express as px
fig = px.histogram(
    coco_ratings_300_df, x="human_grade", facet_row="captionist", facet_col="assessor",
    category_orders={"captionist": ["Conrad", "Aymeric", "Léo", "Arthur"], "assessor": ["Conrad", "Aymeric", "Léo", "Arthur"]}
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

In [ ]:
fig = px.density_heatmap(coco_ratings_300_df, x="captionist", y="assessor", z="human_grade", histfunc='avg')
fig.show()

## Notation des images par CLIP

On donne chaque image à CLIP pour la comparer aux notes données par les évaluateurs humains.

In [ ]:
import pandas as pd

coco_ratings_300_df = pd.read_csv('data/2-coco_df-id-url-captionnist-assessor-grade.csv')

In [ ]:
# reshaping the captions in a 100x3 table

captions_only = coco_ratings_300_df[['caption', 'id']].assign(rank = [1,2,3]*100)
captions_only = captions_only.pivot(values='caption',  index='id', columns='rank')

In [ ]:
import torch
import clip
from PIL import Image

coco_clip_ratings_300_df = coco_ratings_300_df.copy()

verbose = 0
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

for image_id, three_captions in captions_only.iterrows() :
    
    image_path = coco_ratings_300_df.loc[coco_ratings_300_df['id']==image_id, 'filepath']
    image_path = image_path.to_numpy()[0] # normally three lines correspond to one id
    
    if verbose >= 1:
        print( "Image path: ")
        print( image_path )

    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0).to(device)
    text = clip.tokenize( three_captions.to_numpy() ).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features  = model.encode_text(text)
        
        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim = -1).numpy().flatten()

    if verbose >= 1:
        print( "Probs: ")
        print( probs )

    # grades are given three by three
    coco_clip_ratings_300_df.loc[coco_ratings_300_df['id']==image_id, 'clip_grade'] = probs

coco_clip_ratings_300_df.to_csv("data/3-coco_df-clip-grades.csv", index=False)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

## Analyses descriptives

In [ ]:
import pandas as pd

coco_clip_ratings_300_df = pd.read_csv("data/3-coco_df-clip-grades.csv")

In [ ]:
coco_clip_ratings_300_df

,url,captionist,assessor,caption,grade,id,filepath,clip_grade
0,http://cocodataset.org/#explore?id=393274,COCO,Aymeric,a yellow and white train traveling past a trai...,0.9,393274,/root/fiftyone/coco-2017/train/data/0000003932...,7.304041e-01
1,http://cocodataset.org/#explore?id=393274,Arthur,Aymeric,several trains parked in a train station,0.3,393274,/root/fiftyone/coco-2017/train/data/0000003932...,2.695841e-01
2,http://cocodataset.org/#explore?id=393274,Arthur,Aymeric,a river with people nearby,0.0,393274,/root/fiftyone/coco-2017/train/data/0000003932...,1.171632e-05
3,http://cocodataset.org/#explore?id=427123,COCO,Léo,"three people in formal military uniforms, two ...",1.0,427123,/root/fiftyone/coco-2017/train/data/0000004271...,9.990404e-01
4,http://cocodataset.org/#explore?id=427123,Arthur,Léo,smiling researchers receiving the Nobel prize,0.5,427123,/root/fiftyone/coco-2017/train/data/0000004271...,9.263757e-04
...,...,...,...,...,...,...,...,...
295,http://cocodataset.org/#explore?id=541039,Aymeric,Conrad,smiling black people who say thank you,0.9,541039,/root/fiftyone/coco-2017/train/data/0000005410...,1.917687e-04
296,http://cocodataset.org/#explore?id=541039,Aymeric,Conrad,Birthday gifts in a box,0.0,541039,/root/fiftyone/coco-2017/train/data/0000005410...,1.695433e-07
297,http://cocodataset.org/#explore?id=423313,COCO,Arthur,a group of motorcycles parked in front of a bu...,0.9,423313,/root/fiftyone/coco-2017/train/data/0000004233...,9.999363e-01
298,http://cocodataset.org/#explore?id=423313,Aymeric,Arthur,Mortorcyles hiding small stores,0.7,423313,/root/fiftyone/coco-2017/train/data/0000004233...,6.363866e-05


In [ ]:
_deepnote_run_altair(coco_clip_ratings_300_df, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"point","tooltip":{"content":"data"}},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"clip_grade","type":"quantitative","sort":null,"scale":{"type":"sqrt","zero":false}},"y":{"field":"grade","type":"quantitative","sort":null,"scale":{"type":"sqrt","zero":false}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

## Annexes

In [ ]:
# ANNEXE 1
# Code pour générer les répétions d'URLs pour copier-coller dans Google Sheet
import numpy as np
for url in np.repeat( coco_images_100_df.url.to_numpy(), 3):
    print(url)

http://cocodataset.org/#explore?id=393274
http://cocodataset.org/#explore?id=393274
http://cocodataset.org/#explore?id=393274
http://cocodataset.org/#explore?id=427123
http://cocodataset.org/#explore?id=427123
http://cocodataset.org/#explore?id=427123
http://cocodataset.org/#explore?id=405324
http://cocodataset.org/#explore?id=405324
http://cocodataset.org/#explore?id=405324
http://cocodataset.org/#explore?id=300357
http://cocodataset.org/#explore?id=300357
http://cocodataset.org/#explore?id=300357
http://cocodataset.org/#explore?id=57340
http://cocodataset.org/#explore?id=57340
http://cocodataset.org/#explore?id=57340
http://cocodataset.org/#explore?id=198654
http://cocodataset.org/#explore?id=198654
http://cocodataset.org/#explore?id=198654
http://cocodataset.org/#explore?id=518361
http://cocodataset.org/#explore?id=518361
http://cocodataset.org/#explore?id=518361
http://cocodataset.org/#explore?id=525640
http://cocodataset.org/#explore?id=525640
http://cocodataset.org/#explore?id=52

In [ ]:
# ANNEXE 2 Ancien code de Conrad

id_and_pathfile=([(sample['coco_id'],sample['filepath']) for sample in dataset.to_dict()['samples']])

notation_all="""212688	a close up of a polar bear in a body of water;a brown bear bathing in mud;a giraffe running in the toundra
393274	a yellow and white train traveling past a train station;several trains parked in a train station;a river with people nearby
427123	three people in formal military uniforms, two are wearing medals around their necks;smiling researchers receiving the Nobel prize;three donkeys in a dark room
405324	the girl has a plate with a sandwich on it;a fancy dinner downtown;his empty plate causes the baby to cry
300357	a woman in a skirt and colorful shirt kneeling down on floral fabric;a woman trying desperately to leave her bed;two men lying in the sun, one using his dog as a kind of cushion
57340	there is a tower that has a american flag at the top and a clock on it;the night is falling over Manhattan;sunset over the Mississipi river
198654	there are people playing a game of baseball;players play the game;dancers dancing on a Broadway show
518361	three people standing at the zoo watching a elephant;six people standing at the zoo watching two elephant;nobody watching nothing
525640	a banana dressed up to look like a person infront of a paper;a man signing a contract;a salad fruit in the process of making
526347	a skateboarder on a skateboard in a skate park;a young man in shorts doing sports;a girl running after her dog
4130	a bus parked on a street in front of several buildings;a busy street some days before Christmas;a murder scene in a kitchen appliance shop
522661	two people holding suitcases walking towards a greyhound bus;a group of persons running after a departing long-distance bus;two swimmers in bathing suits lining at the front desk
310526	a traffic light with a green signal next to tall buildings;a red traffic light in the country side;a male swan on a lake is courtshipping a female swan
497591	a small child is sitting on a bench outside;a child waiting for his parents to finish shopping;a shop with UNDERGROUND written on the entrance
338991	a man is on a tennis court playing tennis;fours tennis players sharing a court for a match;a very tall man kissing a very short girl
446463	an old fashioned photograph of a fleet of buses;Jules Verne's vision for public transport in 2000;A naked person being captured by the police
50356	a lot of flowers that are sitting on a table;a red rose in a transparent vase;a farmer harvesting sunflowers
301402	a man sits at a table with a laptop computer on his knees;a young boy fidgetting while a woman is talking;a young man asking for helpo in a computer shop
56624	a wood bench is outside covered in snow;a bench decorated for Christmas;three PWC employees writing a report
21097	the meal on the table is ready to be eaten;italian dish and plastic cutlery;a polluted river with dead fish
14824	a group of people dressed as zombies near a stop sign;a confused scene after an attack;a giraffe eating leaves at the top of the trees
317730	a man is holding a surfboard decorated with the image of a woman;a man is holding a surfboard decorated with the image of an octopus;a man is holding skis decorated with snakes
160703	two pizza's on sliver pans on a table with utensils;five pizzas about to be eaten;a mob assaulting an empty pizza place
302313	the thin cow eats some grass next to the tree;black and white cow on a green pasture land;a cow herd mooing under the moonlight
104842	a skateboarded takes to the air in a skateboard park;a skateboarder at rest is watching other skateboarders;a little girl reading a book about skateboards at the library
569865	autumn trees are next to hanging stop lights.;a traffic stop;a man is hitchhiking by the side of road
62824	a street sign stands on the 8500 west block of hollywood boulevard.;a street sign;a traffic light with a green signal next to tall buildings
327802	a street sign set in the edge of the sidewalk that says stockton st.;a street sign on the sidewalk;a picture of the figure 13 from a textbook
385341	a couple of people that are sitting down with wii remotes;two persons holding remotes;some family pictures
370121	a bench sits in a field by the water while it is cloudy.;a bench next to a tree;a family picnic
475277	a white horse looking up for a photo at a fence side.;a close up of a white horse;me riding a white horse
140122	a computer monitor, keyboard, and tower with peripherals and plugs sit on a desk.;a desktop for remote working;a kitchen with a computer
119502	toddler boy in a tie playing in a dish of mud.;a little boy playing outside;a little boy is burying his toy
413868	a man in the woods throws a frisbee;a cowboy throws a bone at his dog;a cowboy is skeet shooting
12650	several people gathered outside of a bar at night.;a group of people in front of a bar;the last bar before the apocalypse
269338	a baseball player sliding into a base with another player bent over with glove extended toward a flying baseball.;a baseball match;two tennismen playing in a court
514601	the train platform has green benches on it.;the train platform is empty;the Massy-Palaiseau train station
495456	a person using an oven mitt to take something from the oven.;something is baking in the oven;someone is baking in the oven
106513	bus at rainy bus stop with commuters waiting;commuters are waiting for the bus;students are waiting for the 91.06 bus
380258	a pigeon sitting outside a window of a high rise building;a pigeon is looking through the window;a peeping tom is looking through the window
61354	an open market selling assorted fruits with bananas hanging;bananas are hanging on a pipe;people are distributing free bananas
506034	there is a side view mirror and traffic lights in this picture;a left view from a vehicule;someone is driving to school
239593	a black and white picture of a train going down a track with a city in the background;a building with many street tags;the train is going through the building
103375	a bed has its covers pulled back in a dark room.;a bedroom with a king size bed;the living room
452687	a person's foot strapped into a black snowboard.;a black snowboard;someone is sunbathing
430885	a cloudy skyline with silhouettes of industrial buildings.;many industrial buildings with a green light;A bansky art
292152	a toasted ham sandwich with a pickle on a white plate.;a fine dining breakfast;a baby is crying because the plate is empty
90024	a giant teddy bear in front of some trees at the mall;a front view of a mall entrance;santa claus is delivering gifts to children
575331	an octopus kite is being flown next to a scuba diver kite above a city park.;two kites, one city park;a airballoon has taken off
404917	a baseball player throwing a baseball on a field.;a baseball game;a boy has broken a window with his baseball thrown
13176	a man bending to look into a small fridge;a man near a fridge;an octopus in the bottom of a hole
230227	a cat sitting on a table with a blurred image in the background;a cat;a dancer and an accordion
122355	four people smiling with tennis rackets in their hands;people playing tennis;a snowman inside a dark forest
359143	a blurry photo of a cyclist on a forest path;a man on a bicycle;the olympic games of tokyo
135288	a young man holding a plate and eating a piece of cake;a man eating a cake;two daughters on the ground
273029	a piece of pizza with many topping on it;a beautiful pizza;an empty field in the setting sun
185633	an old bathroom with peeling paint on the walls and lots of toiletries scattered around;an ugly bathroom;the kitchen of an asiatic restaurant
270222	several small clocks are all connected with human-like ornaments;different clocks on the wall;three ladies looking at burning trees
439868	a skateboarder is jumping onto a rail near crowd;people looking at a skateboarder;a crowd focused on chess players
424174	a tray heaped with fruit including banans apples oranges and limes;a bunch of fruits;an ancient beautiful table and painting on the walls
344720	several cars at an intersection on a city stree;cars near buildings;little red riding hood
292782	a group of people flying their kites on a sunny day;kites in the sky;an old lady eating jam
102765	two young girls sitting on their luggage outside of a brick building;ladies with their luggage;a super-hero fighting bad guys
484149	a field of yellow grass with sheep grazing and standing throughout the areas;sheep grazing;a bunch of flying cows
27593	a man flying through the air on top of a skateboard;a skateboard in the airs;a bed covered in packets of crisps
266165	a cat standing in one food dish while eating out of another dish;a cat eating food;an open window that allows to see the garden
125070	a blue clock pole outside an older building;a clock in the street;two gentlement arguing near a stop sign
78771	a cat sitting next to a laptop computer on a table;a cat on a computer;a baby crying in the grass
291606	a woman standing outside of a bus with asian characters written on it;a bus in Asia;the great pyramids of egypt
423116	on a kitchen counter are a laptop, candy, and cooking supplies;the table of a kitchen with a latpot on it;a sky filled with planes
405762	a man is running while carrying a football;a man playing football;a postman with red hairs in the street
41001	two adult elephants and one baby elephant walking in a grassy area;elephants in the Savannah;A blue cow eating milk
201925	something baking in a pan in the oven and rising out of the pan;a baking cake;the painting of a wedding
380734	several diners eating at a riverside outdoor restaurant;people having dinner in the street;two wolves howling in the snow
487462	a boy is preparing to toss a frisbie while another boy is sitting in the background in a park;a man playing frisbie;a nice looking icecream
455776	a young boy is preparing to catch a baseball.;A human trying to catch a ball;A bottle of wine in a grey field
440895	a white horse can be seen in the window of a brick building.;A black horse in a modern town;French fries playing tennis in a brown field
55627	a bathroom with a mirror above the sink, a towel rack and toilet.;A bathroom with a sink;Picture of a garden full of flowers
436111	a white bathroom has dirt all over the floor.;photograph of a room in low angle;painting of trees 
492286	a big train is about to leave and go away.;A vehicle with people next to it;A tall person standing in front of a door
149892	a bench and chair are seen on the bank of a river.;People sitting next to a river;The floor is lava
493196	a man with helmet skating past some bicycle riders on the road;Bicycle riders under trees;A yellow road with two dogs 
182575	this is a plate of broccoli and cauliflower;A plate of pasta;Three tables and a spoon 
203559	a man is inspecting a baby elephants eye;Three elephants around a man;A black and blue dress
519950	a man is in mid air over a tennis court.;A man jumping in a pool to hit a ball;A dog playing with his owner
27276	a dog is under the bed eating a box;a cat hiding is a small place;A blue and white rollercoaster 
108425	a man tossing a neon green frisbee in front of a house;A nice building with two tables and a chair ;Some grass in a grey background
158713	a man who is driving a motorcycle pulling a cart with two people.;Photograph of a red motorcycle and a white trailer;Three people walking in circles
5701	a person standing on a small paved square and holding a yellow frisbee;A yellow shirt and a yellow frisbee with a green background;The sun shines and the place is beautiful 
254994	a man holding a drinking glass and smiling.;A person with a black tie ;Photograph of a sad dog drinking coffee 
256799	a chandelier sitting above a spiral stair case.;a chandelier and a clock in a very big room with stairs;Two people taking a nap
524981	a dog sitting on a boat behind the sail.;An engine, a dog and a sail in water;Bowling pins going down a slide
509839	a public bathroom that is empty and not in use.;Three sinks in a green room;Two beds in a room
194120	a cat sitting on a windowsill next to a painted pumpkin.;A cat lying on its back eats a pumpkin ;A banana filled with orange jelly 
197329	some zebras and a giraffe tall brown grass and trees;A desert, dry grass and savannah animals;pikachu uses his thunder attack
432637	a group of people riding a carousel with trees in the background.;People having fun on horses;many candlelit sticks
298014	a large architectural structure near a park;a traffic light on a city side walk with people;lighted street lamps
250225	a display of fruit print dishes in a store;Fruits and vegetables in a store;beef steaks spread on the floor
484457	a large stack of square shaped cakes sitting on a lit table.;Two tables and a cake;black sculptures in the shape of chocolate bars
541039	a group of people holding umbrellas standing behind a sign for a umbrella drive.;smiling black people who say thank you;Birthday gifts in a box
423313	a group of motorcycles parked in front of a building.;Mortorcyles hiding small stores;sunflowers looking towards the sun"""

# from PIL import Image

# id_and_pathfile=([(sample['coco_id'],sample['filepath']) for sample in dataset.to_dict()['samples']])
# print(id_and_pathfile)
# liste_de_valeurs= [e.split("\t") for e in notation_all.split("\n")]
# for _,v in enumerate(liste_de_valeurs):
    # cocoid=int(v[0])
    #print(cocoid,list_prompt)
    # a= [valeur[1] for valeur in id_and_pathfile if valeur[0]==cocoid]
    if a:
        # urlcoco= a[0]
        # list_prompt=v[1].split(';')
        # im = Image.open(urlcoco)
        
        # device = "cuda" if torch.cuda.is_available() else "cpu"
        # model, preprocess = clip.load("ViT-B/32", device=device)

        # image = preprocess(im).unsqueeze(0).to(device)
        # text = clip.tokenize(list_prompt).to(device)

        # with torch.no_grad():
        #    image_features = model.encode_image(image)
        #    text_features = model.encode_text(text)
        #    
        #    logits_per_image, logits_per_text = model(image, text)
        #    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

        print("idcoco",cocoid,"Label probs:", probs)
    else:
        print("noimage for :",cocoid)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f3692ed-5f27-49a4-899a-82a03e72232c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>